In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### 1단계

In [3]:
# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [4]:

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()
print(type(documents[0]),documents[0].page_content[:20])

<class 'langchain_core.documents.base.Document'> 제12조(비과세소득) 다음 각 호의 


In [5]:

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
print(len(split_docs),split_docs[11])

12 page_content='아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득
    　　　　1) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금
    　　　　2) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대
    　　　　3) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액
    　　　　4) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액
    　　　　5) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 얻는 이익
    자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당' metadata={'source': 'data/taxinfo.txt'}


In [6]:

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")
print(vectorstore)

In [7]:

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print(type(retrieved_docs))
pprint(retrieved_docs)


<class 'list'>
[Document(id='83992d86-010d-4dcf-bf22-f43742e9b1c0', metadata={'source': 'data/taxinfo.txt'}, page_content='제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득'),
 Document(id='25436e1a-039d-44bd-99d4-37784b74c063', metadata={'source': 'data/taxinfo.txt'}, page_content='가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n    다. 대통령령으로 정하는 농어가부업소득\n    라. 대통령령으로 정하는 전통주의 제

In [8]:

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(context)

제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득
    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득

가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득
    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.
    다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.
    바. 대통령령으로 정하는 작물재배업에서 발생하는 소득

마. 「고용보험법」에 따라 받는 실업급여, 육아휴직 급여, 육아기 근로시간 단축 급여, 출산전후휴가 급여등, 「제대군인 지원에 관한 법률」에 

In [9]:

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)


context 적용한 결과
('소득세법 제12조에 따라 비과세소득에 해당하는 소득은 다음과 같습니다.\n'
 '\n'
 '1. 「공익신탁법」에 따른 공익신탁의 이익\n'
 '2. 사업소득 중 비과세 대상:\n'
 '   - 논밭을 작물 생산에 이용하여 발생하는 소득\n'
 '   - 1주택 소유자의 주택임대소득 (단, 기준시가 12억원 초과 주택 및 해외 주택 제외) 혹은 해당 과세 기간에 총수입금액이 '
 '2천만원 이하인 자의 주택임대소득 (2018년 12월 31일 이전 과세기간 한정)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주 제조 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목 벌채 또는 양도로 인한 소득 (연 600만원 이하)\n'
 '   - 대통령령으로 정하는 작물재배업 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '3. 근로소득과 퇴직소득 중 비과세 대상:\n'
 '   - 대통령령으로 정하는 복무 중인 병(兵)의 급여\n'
 '   - 법률에 따라 동원된 자의 급여\n'
 '   - 「산업재해보상보험법」에 따른 요양급여, 휴업급여, 장해급여 등과 같은 급여\n'
 '   - 「근로기준법」 또는 「선원법」에 따른 요양보상금, 휴업보상금 등 여러 보상금 및 장의비\n'
 '\n'
 '이 외에도 소득세법 제12조에서 명시된 특정 소득은 법령에 따라 비과세됩니다. 각 항목은 세부적으로 특정 법률 및 대통령령에 따르며, '
 '이는 관련 법령을 통해 자세히 확인할 수 있습니다.')


In [10]:

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용하지 않은 결과
('소득세법에서 비과세소득에 해당하는 소득은 과세되지 않는 소득 유형을 의미합니다. 이는 국가별로 다르며, 다음은 일반적으로 한국의 '
 '소득세법에서 비과세로 인정되는 소득의 예시입니다. \n'
 '\n'
 '1. **복지 관련 소득**: 국민건강보험, 고용보험, 국민연금 등의 사회보험금 및 산재보상금.\n'
 '\n'
 '2. **저축성 보험의 보험금**: 일정 요건을 충족하는 저축성 보험의 보험금.\n'
 '\n'
 '3. **군인 등의 급여**: 현역복무 중인 군인, 경찰, 소방 공무원 등의 급여 중 일부.\n'
 '\n'
 '4. **장학금 및 학자금**: 교육 목적으로 지급받는 장학금과 학자금 대출.\n'
 '\n'
 '5. **일정 금액 이하의 이자 및 배당소득**: 소액의 이자 및 배당소득은 비과세되거나 저율로 분리과세될 수 있음.\n'
 '\n'
 '6. **사회통념상 인정되는 경조사비**: 결혼, 장례 등 사회적 관습에 따라 지급받는 일부 금액.\n'
 '\n'
 '7. **일부 해외근로자 소득**: 특정 조건을 만족하는 해외근로자의 근로소득.\n'
 '\n'
 '8. **주택 관련 소득**: 일정 조건 하에 주택을 임대하여 얻는 소득, 특히 소형주택의 임대소득.\n'
 '\n'
 '이 외에도 다양한 비과세 항목이 있을 수 있으므로, 구체적인 사항은 최신 법령이나 전문 세무사와 상담하여 확인하는 것이 좋습니다. 법령은 '
 '주기적으로 개정될 수 있기 때문에 최신 정보를 참고하는 것이 중요합니다.')


### 2단계

In [11]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

개선된 결과:
('소득세법 제12조에 따르면 비과세소득은 다음과 같이 구분됩니다:\n'
 '\n'
 '1. **공익신탁 이익**: 공익신탁법에 따른 공익신탁의 이익(제12조 1호).\n'
 '\n'
 '2. **사업소득**: \n'
 '   - 작물 생산을 위한 논밭 임대 소득(제12조 2호 가목).\n'
 '   - 1주택 소유자의 주택임대소득(기준시가 12억 초과 및 국외 주택 제외, 2018년 이전 소득 한정)(제12조 2호 나목).\n'
 '   - 조림기간 5년 이상 임지의 임목 벌채/양도 소득(연 600만원 이하)(제12조 2호 마목).\n'
 '\n'
 '3. **근로소득 및 퇴직소득**:\n'
 '   - 국외 및 북한지역 근로 급여(제12조 3호 거목).\n'
 '   - 보험료(국가, 지자체, 사용자 부담)(제12조 3호 너목).\n'
 '   - 사내급식 및 월 20만원 이하 식사대(제12조 3호 러목).\n'
 '\n'
 '4. **기타소득**:\n'
 '   - 국가유공자 보훈급여금(제12조 5호 가목).\n'
 '   - 상훈법에 따른 부상 및 상금(제12조 5호 다목).\n'
 '   - 국가지정문화유산 양도 소득(제12조 5호 바목).\n'
 '\n'
 '각 항목은 관련 법령 및 대통령령에 따라 구체적인 조건이 정해집니다.')


### 3단계

In [12]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("./db/faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  # 2 → 6으로 증가
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")


=== 원본 문서 길이 ===
전체 문서 길이: 4971 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금
    저. 대통령령으로 정하는 복리후생적 성질의 급여
4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용

In [13]:

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== MMR 검색 결과 ===
('소득세법 제12조에 따른 비과세소득 항목들을 다음과 같이 체계적으로 정리할 수 있습니다:\n'
 '\n'
 '1. **공익신탁의 이익**\n'
 '   - 「공익신탁법」에 따른 공익신탁의 이익\n'
 '\n'
 '2. **사업소득**\n'
 '   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 '
 '제외)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '\n'
 '3. **근로소득과 퇴직소득**\n'
 '   - 대통령령으로 정하는 복무 중인 병이 받는 급여\n'
 '   - 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 수급권자가 받는 각종 보상금\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 각종 보상금\n'
 '   - 대통령령으로 정하는 복리후생적 성질의 급여\n'
 '\n'
 '4. **연금소득**\n'
 '   - 공적연금 관련법에 따라 받는 유족연금, 장애연금 등\n'
 '   - 「산업재해보상보험법」에 따라 받는 각종 연금\n'
 '   - 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금\n'
 '\n'
 '5. **기타소득**\n'
 '   - 「국가유공자 등 예우 및 지원에 관한 법률」에 따라 받는 보훈급여금 등\n'
 '   - 「국가보안법」에 따라 받는 상금과 보로금\n'
 '   - 「상훈법」에